In [1]:
from typing import List, Optional, Union, Literal
from pydantic import BaseModel, Field
import requests


class Message(BaseModel):
    role: Literal["system", "user", "assistant", "developer"]
    content: str


class KBFilter(BaseModel):
    index: str
    path: Optional[str] = None


class ChatCompletionRequest(BaseModel):
    messages: List[Message]
    temperature: Optional[float] = Field(default=None, ge=0.0, le=2.0)
    top_p: Optional[float] = Field(default=None, ge=0.01)
    max_tokens: Optional[int] = Field(default=None, ge=1)
    max_completion_tokens: Optional[int] = Field(default=None, ge=1)
    stream: Optional[bool] = False
    k: Optional[int] = Field(default=None, ge=0)
    retrieval_method: Optional[Literal["rewrite", "step_back", "sub_queries", "none"]] = None
    frequency_penalty: Optional[float] = Field(default=0.0, ge=-2.0, le=2.0)
    presence_penalty: Optional[float] = Field(default=0.0, ge=-2.0, le=2.0)
    stop: Optional[Union[str, List[str]]] = None
    instruction_override: Optional[str] = None
    include_functions_info: Optional[bool] = False
    include_retrieval_info: Optional[bool] = False
    include_guardrails_info: Optional[bool] = False
    provide_citations: Optional[bool] = False
    kb_filters: Optional[List[KBFilter]] = None
    filter_kb_content_by_query_metadata: Optional[bool] = False


class GenAIAgentClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url.rstrip('/')
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

    def health_check(self) -> dict:
        url = f"{self.base_url}/health"
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        return response.json()

    def chat_completion(self, payload: ChatCompletionRequest) -> dict:
        url = f"{self.base_url}/api/v1/chat/completions"
        response = requests.post(url, headers=self.headers, json=payload.model_dump(exclude_none=True))
        response.raise_for_status()
        return response.json()


In [2]:
text = '«Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. Борха Иглесиас забил хет-трик за «Сельту», но «Барселона» смогла отыграться благодаря голам Дани Ольмо, Рафиньи и Ламина. Победный гол был забит Рафиньей на 90+6-й минуте с пенальти. Эта победа позволила «Барселоне» увеличить преимущество над «Реалом» в чемпионате до 7 очков.'

text

'«Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. Борха Иглесиас забил хет-трик за «Сельту», но «Барселона» смогла отыграться благодаря голам Дани Ольмо, Рафиньи и Ламина. Победный гол был забит Рафиньей на 90+6-й минуте с пенальти. Эта победа позволила «Барселоне» увеличить преимущество над «Реалом» в чемпионате до 7 очков.'

In [ ]:
system_prompt = {
    "en": """You are a news classification assistant.

Your task is to:
1. Determine the most relevant top-level category for the given news article.
2. Select the 3 most appropriate subcategories within that category.
3. Make a summary of the text, keeping only the key facts, important statements and significant actions. Keep the political and emotional context. Keep it brief but to the point.
4. Importance of news, 0 - news does not affect the last number of people, 1 - news can affect the last number of people, 2 - news directly affects a large number of people
5. Return the result strictly in JSON format like:

{
  "category": "main category",
  "subcategories": ["subcategory1", "subcategory2", "subcategory3"],
  "importance": 0,
  "summary": "article summary",
}

Here are the available categories and their subcategories:

""",
    "ru": """
Ты — интеллектуальный ассистент по классификации новостей.
Твоя задача:
1. Определи наиболее подходящую **основную категорию** новости (например: "спорт", "экономика", "культура" и т.д.).
2. Внутри этой категории выбери **3 наиболее релевантных подкатегории**, которые лучше всего соответствуют содержанию новости.
3. Сделай краткое содержание текста, сохранив только ключевые факты, важные высказывания и значимые действия. Сохраняй политический и эмоциональный контекст. Приводи кратко, но ёмко. 
4. Важность новости, 0 - новость не влияет на значительное кол-во людей, 1  - новость может повлиятель на значительное кол-во людей, 2  - новость на прямую затрагивает большое кол-во людей
5. Извлеки список компаний которые упомянались в тексте
6. Извлеки список мест или стран или городов которые упомянались в тексте
7. Извлеки список имен людей которые упомянались в тексте
8. Верни результат строго в формате JSON, **без объяснений**, например:

{
  "category": "категория",
  "subcategories": ["подкатегория1", "подкатегория2", "подкатегория3"],
  "importance": 0,
  "summary": "краткое содержание статьи",
  "companies: ["компания1", "компания2"],
  "locations: ["место", "место"],
  "names: ["имя1", "имя2"],
}

Вот список всех возможных категорий и подкатегорий:

"""
}


In [1]:
d = {
    "политика": {
        "hypothesis": "Этот текст связан с политикой, правительством, выборами или международными отношениями.",
        "items": {
            "выборы": "Этот текст о выборах, кандидатах или избирательной кампании.",
            "санкции": "Этот текст касается экономических или политических санкций между странами.",
            "коррупция": "Этот текст связан с коррупцией, злоупотреблением властью или расследованиями.",
            "международные отношения": "Этот текст описывает отношения между странами, дипломатию или внешнюю политику.",
            "госуправление": "Этот текст о государственном управлении, министерствах или чиновниках.",
            "законопроекты": "Этот текст касается законопроектов, обсуждений в парламенте или законодательных инициатив.",
            "военные конфликты": "Этот текст о вооружённых конфликтах, войнах или военных операциях."
        }
    },
    "экономика": {
        "hypothesis": "Этот текст касается экономики, макроэкономики, инфляции или финансовых показателей страны.",
        "items": {
            "инфляция": "Этот текст о росте цен, инфляции или покупательной способности.",
            "безработица": "Этот текст касается безработицы, рынка труда или занятости населения.",
            "налоги": "Этот текст о налогах, налоговой политике или изменениях в налогообложении.",
            "бюджет": "Этот текст касается государственного бюджета, расходов или финансового планирования.",
            "валюта": "Этот текст о валютных курсах, обмене валют или денежной политике.",
            "госдолг": "Этот текст о государственном долге, заимствованиях или долговых обязательствах.",
            "макроэкономика": "Этот текст о макроэкономических показателях и общей экономической ситуации в стране."
        }
    },
    "бизнес": {
        "hypothesis": "Этот текст связан с деловой активностью, компаниями, рынками или стартапами.",
        "items": {
            "рынки": "Этот текст касается фондовых, товарных или криптовалютных рынков.",
            "сделки": "Этот текст о сделках между компаниями, приобретениях или слияниях.",
            "финансовая отчётность": "Этот текст о финансовых результатах компаний или корпоративной отчётности.",
            "IPO": "Этот текст о первичном размещении акций компаний на бирже (IPO).",
            "корпорации": "Этот текст о крупных компаниях, корпорациях и их деятельности.",
            "предпринимательство": "Этот текст касается малого бизнеса, стартапов или индивидуальных предпринимателей.",
            "банковский сектор": "Этот текст о банках, кредитах, банковской системе или Центробанке."
        }
    },
    "технологии": {
        "hypothesis": "Этот текст о технологиях, цифровых продуктах, IT-индустрии или инновациях.",
        "items": {
            "искусственный интеллект": "Этот текст о нейросетях, машинном обучении или искусственном интеллекте.",
            "стартапы": "Этот текст о технологических стартапах, инвестициях или акселераторах.",
            "гаджеты": "Этот текст о новых устройствах, смартфонах, ноутбуках или других гаджетах.",
            "кибербезопасность": "Этот текст о цифровой безопасности, хакерских атаках или защите данных.",
            "социальные сети": "Этот текст касается соцсетей, их обновлений, трендов или влияния на общество.",
            "блокчейн": "Этот текст касается технологии блокчейн, криптовалют или децентрализации.",
            "робототехника": "Этот текст о роботах, автоматизации или технологических разработках в области механики."
        }
    },
    "спорт": {
        "hypothesis": "Этот текст описывает спортивные события, соревнования, команды или спортсменов.",
        "items": {
            "футбол": "Этот текст о футбольных матчах, клубах, игроках или турнирах.",
            "олимпийские игры": "Этот текст касается Олимпийских игр, спортсменов и дисциплин.",
            "теннис": "Этот текст о теннисных турнирах, рейтингах и игроках.",
            "хоккей": "Этот текст о хоккейных матчах, лигах, командах и событиях.",
            "трансферы": "Этот текст о переходах спортсменов между клубами и трансферных сделках.",
            "чемпионаты": "Этот текст касается национальных и международных спортивных чемпионатов.",
            "рекорды": "Этот текст о спортивных достижениях и рекордах."
        }
    },
    "общество": {
        "hypothesis": "Этот текст о социальных темах, правах человека, демографии или жизни общества в целом.",
        "items": {
            "образование": "Этот текст касается школ, вузов, образовательных реформ или обучения.",
            "миграция": "Этот текст о миграции, беженцах или миграционной политике.",
            "семья": "Этот текст о семейной политике, родительстве или семейных ценностях.",
            "трудоустройство": "Этот текст о поиске работы, карьерных возможностях или занятости.",
            "социальная политика": "Этот текст о социальной поддержке, пособиях или социальной защите населения.",
            "молодёжь": "Этот текст касается вопросов, связанных с молодыми людьми, их образованием, досугом или инициативами.",
            "демография": "Этот текст касается населения, рождаемости, смертности или возрастной структуры."
        }
    },
    "происшествия": {
        "hypothesis": "Этот текст о несчастных случаях, авариях, катастрофах или уголовных происшествиях.",
        "items": {
            "пожары": "Этот текст о пожарах, возгораниях и их последствиях.",
            "ДТП": "Этот текст касается дорожно-транспортных происшествий и аварий.",
            "преступления": "Этот текст о преступлениях, уголовных делах или расследованиях.",
            "наводнения": "Этот текст о наводнениях, подтоплениях или природных катастрофах.",
            "теракты": "Этот текст касается террористических актов или угроз.",
            "взрывы": "Этот текст о взрывах, утечках газа или техногенных авариях.",
            "катастрофы": "Этот текст о крупных техногенных или природных катастрофах."
        }
    },
    "культура": {
        "hypothesis": "Этот текст посвящён культуре, искусству, литературе, кино или театру.",
        "items": {
            "кино": "Этот текст касается кинофильмов, премий, актёров или новинок кино.",
            "литература": "Этот текст о книгах, писателях, литературных премиях или литературных обзорах.",
            "театр": "Этот текст о театральных постановках, актёрах и театральной жизни.",
            "музыка": "Этот текст о музыке, концертах, исполнителях или музыкальных релизах.",
            "живопись": "Этот текст касается изобразительного искусства, художников или выставок.",
            "фестивали": "Этот текст о культурных фестивалях, мероприятиях или праздниках.",
            "музеи": "Этот текст касается музеев, экспозиций и историко-культурного наследия."
        }
    },
    "наука": {
        "hypothesis": "Этот текст связан с научными открытиями, исследованиями, университетами или технологиями будущего.",
        "items": {
            "астрофизика": "Этот текст касается космоса, звёзд, планет и астрофизических исследований.",
            "биология": "Этот текст о живых организмах, эволюции или биологических открытиях.",
            "медицина": "Этот текст о медицинских исследованиях, терапии, здоровье и лечении.",
            "экология": "Этот текст касается окружающей среды, защиты природы или климата.",
            "инновации": "Этот текст о новейших научных разработках и прорывах.",
            "исследования": "Этот текст о научных исследованиях и экспериментах.",
            "технологические открытия": "Этот текст о новых научных и технологических достижениях."
        }
    },
    "авто": {
        "hypothesis": "Этот текст касается автомобилей, автопрома, новых моделей или транспортной инфраструктуры.",
        "items": {
            "электромобили": "Этот текст о электромобилях, зарядках и развитии электротранспорта.",
            "дорожные тесты": "Этот текст о тест-драйвах, обзорах и оценке автомобилей.",
            "новые модели": "Этот текст о выходе новых моделей автомобилей и их характеристиках.",
            "аварии": "Этот текст о дорожно-транспортных происшествиях с участием автомобилей.",
            "автосалон": "Этот текст о выставках автомобилей, автосалонах и премьерах.",
            "тюнинг": "Этот текст о модификациях автомобилей и кастомизации.",
            "грузовики": "Этот текст о грузовых автомобилях, логистике и технике."
        }
    },
    "медиа": {
        "hypothesis": "Этот текст о СМИ, журналистике, телевидении, блогерах или информационных платформах.",
        "items": {
            "телевидение": "Этот текст касается телешоу, каналов, телепередач и телевещания.",
            "интернет-СМИ": "Этот текст о новостных онлайн-изданиях, блогах и цифровых медиа.",
            "журналистика": "Этот текст касается работы журналистов, СМИ и свободы прессы.",
            "цензура": "Этот текст касается ограничений, блокировок или контроля информации.",
            "социальные медиа": "Этот текст о платформах вроде Instagram, VK, YouTube и TikTok.",
            "трансляции": "Этот текст о прямых эфирах, стриминге и видеотрансляциях.",
            "инфлюенсеры": "Этот текст о блогерах, медийных личностях и их влиянии."
        }
    },
    "здоровье": {
        "hypothesis": "Этот текст связан с медициной, здоровьем, профилактикой болезней или здравоохранением.",
        "items": {
            "психология": "Этот текст касается психического здоровья, стресса или поведенческой терапии.",
            "заболевания": "Этот текст о болезнях, симптомах, профилактике или лечении.",
            "лекарства": "Этот текст о медицинских препаратах, вакцинах и лечении заболеваний.",
            "здоровый образ жизни": "Этот текст о питании, фитнесе и профилактике заболеваний.",
            "медицина": "Этот текст о системе здравоохранения, врачах и медицинских услугах.",
            "эпидемии": "Этот текст о вспышках инфекционных заболеваний и их распространении.",
            "медуслуги": "Этот текст о доступности и качестве медицинских услуг и учреждений."
        }
    },
    "путешествия": {
        "hypothesis": "Этот текст посвящён туризму, путешествиям, странам, городам и рекомендациям для туристов.",
        "items": {
            "туризм": "Этот текст о туристических направлениях, турах и путешествиях.",
            "авиаперелёты": "Этот текст о перелётах, авиакомпаниях и воздушном транспорте.",
            "отели": "Этот текст об отелях, бронировании и гостиничном бизнесе.",
            "визы": "Этот текст о визовой политике, получении виз и документах для путешествий.",
            "круизы": "Этот текст о морских путешествиях, лайнерах и круизах.",
            "гиды": "Этот текст о путеводителях, экскурсоводах и туристических маршрутах.",
            "страны и города": "Этот текст о странах, городах и достопримечательностях."
        }
    }
}
d3 = {
    "politics": {
        "hypothesis": "This text is related to politics, government, elections, or international relations.",
        "items": {
            "elections": "This text is about elections, candidates, or political campaigns.",
            "sanctions": "This text concerns economic or political sanctions between countries.",
            "corruption": "This text is about corruption, abuse of power, or investigations.",
            "international relations": "This text discusses relationships between countries, diplomacy, or foreign policy.",
            "governance": "This text is about public administration, ministries, or government officials.",
            "legislation": "This text is about legislative initiatives, bills, or parliamentary debates.",
            "military conflicts": "This text is about armed conflicts, wars, or military operations."
        }
    },
    "economy": {
        "hypothesis": "This text relates to economics, macroeconomics, inflation, or national financial indicators.",
        "items": {
            "inflation": "This text is about price increases, inflation, or purchasing power.",
            "unemployment": "This text concerns unemployment, labor markets, or job statistics.",
            "taxes": "This text is about taxation, tax policy, or changes in the tax system.",
            "budget": "This text is about the state budget, government spending, or financial planning.",
            "currency": "This text is about currency exchange rates or monetary policy.",
            "public debt": "This text is about national debt, borrowings, or financial obligations.",
            "macroeconomics": "This text deals with macroeconomic indicators and the overall economic climate."
        }
    },
    "business": {
        "hypothesis": "This text is related to business activities, companies, markets, or startups.",
        "items": {
            "markets": "This text is about stock, commodity, or cryptocurrency markets.",
            "deals": "This text is about corporate deals, mergers, or acquisitions.",
            "financial reporting": "This text covers companies’ financial results or earnings reports.",
            "IPO": "This text is about initial public offerings (IPOs) and public listings.",
            "corporations": "This text is about large corporations and their operations.",
            "entrepreneurship": "This text is about small businesses, startups, or individual entrepreneurs.",
            "banking": "This text is about banks, lending, financial institutions, or central banking."
        }
    },
    "technology": {
        "hypothesis": "This text discusses technology, digital products, the IT industry, or innovation.",
        "items": {
            "artificial intelligence": "This text is about AI, neural networks, or machine learning.",
            "startups": "This text is about tech startups, investment rounds, or incubators.",
            "gadgets": "This text covers new devices like smartphones, laptops, or wearables.",
            "cybersecurity": "This text is about digital security, data protection, or cyberattacks.",
            "social media": "This text concerns platforms like Instagram, TikTok, or Facebook.",
            "blockchain": "This text is about blockchain technology, cryptocurrencies, or decentralization.",
            "robotics": "This text is about robots, automation, and mechanical engineering innovations."
        }
    },
    "sports": {
        "hypothesis": "This text covers sports events, competitions, teams, or athletes.",
        "items": {
            "football": "This text is about football games, players, clubs, or tournaments.",
            "olympics": "This text relates to the Olympic Games, athletes, and events.",
            "tennis": "This text is about tennis tournaments, rankings, or players.",
            "hockey": "This text discusses hockey games, leagues, teams, or players.",
            "transfers": "This text is about athlete transfers and sports contracts.",
            "championships": "This text is about national or international championships.",
            "records": "This text is about sports achievements and record-breaking performances."
        }
    },
    "society": {
        "hypothesis": "This text is about societal topics, human rights, demographics, or social life.",
        "items": {
            "education": "This text is about schools, universities, reforms, or education systems.",
            "migration": "This text concerns immigration, refugees, or migration policy.",
            "family": "This text is about family policy, parenting, or family life.",
            "employment": "This text relates to job opportunities, hiring, or career development.",
            "social policy": "This text is about social support, benefits, or government aid programs.",
            "youth": "This text covers youth-related issues such as education, culture, or activism.",
            "demographics": "This text concerns population changes, birth rates, or age distribution."
        }
    },
    "incidents": {
        "hypothesis": "This text discusses accidents, emergencies, disasters, or criminal events.",
        "items": {
            "fires": "This text is about fires, wildfires, or firefighting.",
            "traffic accidents": "This text is about traffic incidents, crashes, or road safety.",
            "crimes": "This text is about criminal cases, law violations, or police investigations.",
            "floods": "This text is about floods, water damage, or natural disasters.",
            "terrorist acts": "This text discusses terrorism or related threats.",
            "explosions": "This text is about explosions, gas leaks, or industrial accidents.",
            "catastrophes": "This text is about major disasters, whether natural or man-made."
        }
    },
    "culture": {
        "hypothesis": "This text is about culture, arts, literature, cinema, or theater.",
        "items": {
            "cinema": "This text is about movies, actors, film awards, or premieres.",
            "literature": "This text discusses books, authors, literary prizes, or publishing.",
            "theater": "This text is about theater productions, plays, or performers.",
            "music": "This text covers music, concerts, artists, or releases.",
            "painting": "This text is about visual arts, painters, or exhibitions.",
            "festivals": "This text discusses cultural festivals or public celebrations.",
            "museums": "This text concerns museums, exhibitions, or historical heritage."
        }
    },
    "science": {
        "hypothesis": "This text is related to scientific discoveries, research, universities, or futuristic technologies.",
        "items": {
            "astrophysics": "This text is about space, stars, planets, and astrophysical discoveries.",
            "biology": "This text is about living organisms, evolution, or life sciences.",
            "medicine": "This text covers health research, treatments, or clinical studies.",
            "ecology": "This text concerns environmental protection, climate, or nature.",
            "innovation": "This text is about cutting-edge science and breakthrough developments.",
            "research": "This text is about academic research or scientific studies.",
            "technological discoveries": "This text is about newly discovered technologies or scientific progress."
        }
    },
    "automotive": {
        "hypothesis": "This text is related to cars, automotive industry, new models, or transportation systems.",
        "items": {
            "electric vehicles": "This text discusses electric cars, charging stations, or e-mobility.",
            "road tests": "This text is about car reviews, road tests, or performance assessments.",
            "new models": "This text covers upcoming vehicle releases or specs of new car models.",
            "accidents": "This text is about car crashes, road incidents, or auto-related injuries.",
            "autoshow": "This text is about car expos, auto shows, and public unveilings.",
            "tuning": "This text concerns car modifications and custom designs.",
            "trucks": "This text is about trucks, logistics, and commercial transportation."
        }
    },
    "media": {
        "hypothesis": "This text is about media, journalism, television, influencers, or information platforms.",
        "items": {
            "television": "This text is about TV shows, broadcasts, or channels.",
            "online media": "This text covers digital news outlets, blogs, or websites.",
            "journalism": "This text is about reporters, press freedom, or news reporting.",
            "censorship": "This text concerns censorship, content control, or information suppression.",
            "social platforms": "This text is about social media services like YouTube or TikTok.",
            "live broadcasts": "This text is about live streaming, shows, or real-time events.",
            "influencers": "This text is about bloggers, content creators, or media personalities."
        }
    },
    "health": {
        "hypothesis": "This text is about healthcare, medicine, disease prevention, or well-being.",
        "items": {
            "psychology": "This text is about mental health, stress, or psychological treatment.",
            "diseases": "This text is about illnesses, symptoms, or medical conditions.",
            "medications": "This text is about drugs, vaccines, or medical products.",
            "healthy lifestyle": "This text discusses nutrition, exercise, or wellness habits.",
            "medical system": "This text is about doctors, hospitals, or healthcare services.",
            "epidemics": "This text is about outbreaks, pandemics, or disease spread.",
            "healthcare services": "This text concerns medical care, access, or public health systems."
        }
    },
    "travel": {
        "hypothesis": "This text is about tourism, travel, countries, cities, or tips for travelers.",
        "items": {
            "tourism": "This text is about travel destinations, tours, or vacation planning.",
            "air travel": "This text concerns airlines, flights, or airports.",
            "hotels": "This text is about accommodations, hotels, or reservations.",
            "visas": "This text concerns visa requirements or travel documents.",
            "cruises": "This text is about sea voyages, cruises, or passenger liners.",
            "guides": "This text is about travel guides, excursions, or local tips.",
            "countries and cities": "This text is about countries, cities, and tourist attractions."
        }
    }
}


d1 = []
for k, v in d.items():
    d1.append(
        {
            "category": k,
            "subcategories": [k1 for k1 in v["items"].keys()]
        }
    )


d2 = []
for k, v in d3.items():
    d2.append(
        {
            "category": k,
            "subcategories": [k1 for k1 in v["items"].keys()]
        }
    )

In [6]:
data = {}
i = 1
for k1, k2 in zip(d1, d2):
    data[i] = {
        "id": i,
        "key": k2["category"],
        "parent_id": None,
        
    }
    parrent_id = i
    i += 1
    for v1, v2 in zip(k1["subcategories"], k2["subcategories"]):
        data[i] = {
            "id": i,
            "key": v2,
            "parent_id": parrent_id,
            
        }
        i += 1


In [7]:
[v for v in data.values()]

[{'id': 1, 'key': 'politics', 'parent_id': None},
 {'id': 2, 'key': 'elections', 'parent_id': 1},
 {'id': 3, 'key': 'sanctions', 'parent_id': 1},
 {'id': 4, 'key': 'corruption', 'parent_id': 1},
 {'id': 5, 'key': 'international relations', 'parent_id': 1},
 {'id': 6, 'key': 'governance', 'parent_id': 1},
 {'id': 7, 'key': 'legislation', 'parent_id': 1},
 {'id': 8, 'key': 'military conflicts', 'parent_id': 1},
 {'id': 9, 'key': 'economy', 'parent_id': None},
 {'id': 10, 'key': 'inflation', 'parent_id': 9},
 {'id': 11, 'key': 'unemployment', 'parent_id': 9},
 {'id': 12, 'key': 'taxes', 'parent_id': 9},
 {'id': 13, 'key': 'budget', 'parent_id': 9},
 {'id': 14, 'key': 'currency', 'parent_id': 9},
 {'id': 15, 'key': 'public debt', 'parent_id': 9},
 {'id': 16, 'key': 'macroeconomics', 'parent_id': 9},
 {'id': 17, 'key': 'business', 'parent_id': None},
 {'id': 18, 'key': 'markets', 'parent_id': 17},
 {'id': 19, 'key': 'deals', 'parent_id': 17},
 {'id': 20, 'key': 'financial reporting', 'paren

In [5]:
import json

client = GenAIAgentClient(base_url="https://bm7tmpgxhhbq42xfwdsvlzjf.agents.do-ai.run/", api_key="r3U2WKvdPenZ3hX9dK29tqFIcTyQpl2n")

lang = "ru"
content = system_prompt[lang] + json.dumps(d1, ensure_ascii=False, indent=2)

request_payload = ChatCompletionRequest(
    messages=[
        Message(role="system", content=content),
        Message(role="user", content=text)
    ],
    temperature=0.3,
    top_p=0.9,
    max_tokens=3000,
    max_completion_tokens=500,
)

response = client.chat_completion(payload=request_payload)

In [6]:
print(response["choices"][0]["message"]["content"])

{
  "category": "спорт",
  "subcategories": ["футбол", "чемпионаты", "трансферы"],
  "importance": 1,
  "summary": "«Барселона» совершила камбэк в матче против «Сельты», проигрывая 1:3, но в итоге выиграв 4:3. Борха Иглесиас забил хет-трик за «Сельту», но «Барселона» смогла отыграться благодаря голам Дани Ольмо, Рафиньи и Ламина. Победный гол был забит Рафиньей на 90+6-й минуте с пенальти.",
  "companies": [],
  "locations": ["Барселона"],
  "names": ["Борха Иглесиас", "Дани Ольмо", "Рафинья", "Ламина"]
}


In [40]:
response["usage"]

{'prompt_tokens': 2529, 'completion_tokens': 190, 'total_tokens': 2719}

In [35]:
response["usage"]

{'prompt_tokens': 2440, 'completion_tokens': 29, 'total_tokens': 2469}